In [1]:
import cv2
import numpy as np
import pandas as pd
import pytesseract
import easyocr
import re
from PIL import Image

pytesseract.pytesseract.tesseract_cmd = 'C:/Program Files/Tesseract-OCR/tesseract.exe'
reader = easyocr.Reader(['ru'])

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


In [2]:
path = 'C:/Users/Anastasia/Downloads/'
img_path = path + 'cropped.jpg'

# Итоговая функция

In [3]:
# функция для распознавания таблиц с изображения
# pytesseract для текста + EasyOCR для недостающих (digit=1)
# или + EasyOCR для цифр (digit=1)
def img2table(img_path, output_path, digit=1):
    # Считывание изображения
    img = Image.open(img_path)
    img = np.array(img)
    if img.ndim == 3 and img.shape[2] == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Присвоение изображению порогового значения в виде двоичного изображения
    img_bin = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 21, 10)

    # Ширина ядра как 100-я часть общей ширины
    kernel_len = np.array(img).shape[1] // 100

    # Определение вертикального и горизонтального ядер
    ver_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, kernel_len))
    hor_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_len, 1))

    # Ядро размером 2x2
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))

    # Обнаружение вертикальных и горизонтальных линий
    image_1 = cv2.erode(img_bin, ver_kernel, iterations=3)
    vertical_lines = cv2.dilate(image_1, ver_kernel, iterations=3)

    image_2 = cv2.erode(img_bin, hor_kernel, iterations=3)
    horizontal_lines = cv2.dilate(image_2, hor_kernel, iterations=3)

    # Объединение горизонтальных и вертикальных линий в новом изображении
    img_vh = cv2.addWeighted(vertical_lines, 0.5, horizontal_lines, 0.5, 0.0)

    # Размывание и установление порогового значения
    img_vh = cv2.erode(~img_vh, kernel, iterations=2)
    _, img_vh = cv2.threshold(img_vh, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

    # Обнаружение контуров
    contours, _ = cv2.findContours(img_vh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Преобразование tuple в list
    contours = list(contours)
    contours.reverse()

    # Подсчёт вертикальных линий (столбцов)
    num_vertical_lines = 1
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        
        if w > 20 and h > 20 and w < 500 and h < 500:
            num_vertical_lines = int(img.shape[1] / w)
            break
    
    texts = []
    # Итерация по каждому контуру (ячейке)
    for contour in contours:
        
        # Получение координат ограничивающего прямоугольника вокруг контура
        x, y, w, h = cv2.boundingRect(contour)
        if w > 20 and h > 20 and w < 500 and h < 500:
            cell_image = img[y:y + h, x:x + w]
        
            # Получение текста с ячейки
            text = pytesseract.image_to_string(cell_image, lang='rus')

            if digit == 0: # вариант "EasyOCR для недостающих"
                if text == '':
                    results = reader.readtext(cell_image)
                    if len(results) > 0:
                        text = results[0][1]
            else: # вариант "EasyOCR для цифр"
                if text == '' or re.search(r'\d', text):
                    results = reader.readtext(cell_image)
                    if len(results) > 0:
                        text = results[0][1]

            # Добавление текста в список
            texts.append(text.replace('|', '').replace('_', '').replace(',', '.'))
    
    # Исключение пустых строк вначале списка
    for text in texts[num_vertical_lines:]:
        if text == '':
            texts.remove(text)
        else:
            break
        
    # Разделение списка по колучиству столбцов
    texts = [texts[i:i+num_vertical_lines] for i in range(0, len(texts), num_vertical_lines)]
    
    # Создание датафрейма
    try:
        df = pd.DataFrame(data=texts[1:], columns=texts[0])
        df = df.applymap(lambda x: x.replace('\n', ''))
        df = df.rename(columns=lambda x: re.sub(r'\s+', ' ', x.replace('\n', ' ')))
    except:
        df = pd.DataFrame()
    
    # Сохранение
    df.to_csv(output_path, index=False)

    return df

In [4]:
final_path = path + 'final.csv'

In [5]:
%%time
img2table(img_path, final_path)

CPU times: total: 16 s
Wall time: 7.9 s


,Дата,Вид дон-ва,Кол-во,Дата,Вид дон-ва,Кол-во,Дата,Вид дон-ва,Кол-во
0,14.02.2006,кр/д (бв),420,15.07.2016,кр/д (бв),450,04.08.2018,кр/д (бв),450
1,11.06.2014,кр/д (бв),350,11.10.2016,кр/д (бв),450,26.12.2018,кр/д (бв),450
2,30.10.2014,кр/д (бв),450,21.12.2016,кр/д (бв),450,29.03.2019,кр/д (бв),450
3,13.08.2015,кр/д (бв),450,21.06.2017,кр/д (бв),450,11.10.2022,кр/д (бв),450
4,30.10.2015,кр/д (бв),450,,,,,,


# pytesseract для текста + EasyOCR для цифр

In [6]:
def img2table(img_path, output_path):
    # Считывание изображения
    img = Image.open(img_path)
    img = np.array(img)
    if img.ndim == 3 and img.shape[2] == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Присвоение изображению порогового значения в виде двоичного изображения
    img_bin = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 21, 10)

    # Ширина ядра как 100-я часть общей ширины
    kernel_len = np.array(img).shape[1] // 100

    # Определение вертикального и горизонтального ядер
    ver_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, kernel_len))
    hor_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_len, 1))

    # Ядро размером 2x2
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))

    # Обнаружение вертикальных и горизонтальных линий
    image_1 = cv2.erode(img_bin, ver_kernel, iterations=3)
    vertical_lines = cv2.dilate(image_1, ver_kernel, iterations=3)

    image_2 = cv2.erode(img_bin, hor_kernel, iterations=3)
    horizontal_lines = cv2.dilate(image_2, hor_kernel, iterations=3)

    # Объединение горизонтальных и вертикальных линий в новом изображении
    img_vh = cv2.addWeighted(vertical_lines, 0.5, horizontal_lines, 0.5, 0.0)

    # Размывание и установление порогового значения
    img_vh = cv2.erode(~img_vh, kernel, iterations=2)
    _, img_vh = cv2.threshold(img_vh, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

    # Обнаружение контуров
    contours, _ = cv2.findContours(img_vh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Преобразование tuple в list
    contours = list(contours)
    contours.reverse()

    # Подсчёт вертикальных линий (столбцов)
    num_vertical_lines = 1
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        
        if w > 20 and h > 20 and w < 500 and h < 500:
            num_vertical_lines = int(img.shape[1] / w)
            break
    
    texts = []
    # Итерация по каждому контуру (ячейке)
    for contour in contours:
        
        # Получение координат ограничивающего прямоугольника вокруг контура
        x, y, w, h = cv2.boundingRect(contour)
        if w > 20 and h > 20 and w < 500 and h < 500:
            cell_image = img[y:y + h, x:x + w]
        
            # Получение текста с ячейки
            text = pytesseract.image_to_string(cell_image, lang='rus')

            if text == '' or re.search(r'\d', text):
                results = reader.readtext(cell_image)
                if len(results) > 0:
                    text = results[0][1]
        
            # Добавление текста в список
            texts.append(text.replace('|', '').replace('_', ''))
    
    # Разделение списка по колучиству столбцов
    texts = [texts[i:i+num_vertical_lines] for i in range(0, len(texts), num_vertical_lines)]
    
    # Создание датафрейма
    try:
        df = pd.DataFrame(data=texts[1:], columns=texts[0])
    except:
        df = pd.DataFrame(columns=texts)
    df.dropna(inplace=True)
    df = df.applymap(lambda x: x.replace('\n', ''))
    df = df.rename(columns=lambda x: re.sub(r'\s+', ' ', x.replace('\n', ' ')))
    
    # Сохранение
    df.to_csv(output_path, index=False)

    return df

In [7]:
easy_dig_path = path + 'easy_dig.csv'

In [8]:
%%time
img2table(img_path, easy_dig_path)

CPU times: total: 16.9 s
Wall time: 7.68 s


,Дата,Вид дон-ва,Кол-во,Дата,Вид дон-ва,Кол-во,Дата,Вид дон-ва,Кол-во
0,,,,,,,,,
1,14.02.2006,кр/д (бв),420,15.07.2016,кр/д (бв),450,04.08.2018,кр/д (бв),450
2,11.06.2014,кр/д (бв),350,11.10.2016,кр/д (бв),450,"26.12,2018",кр/д (бв),450
3,30.10.2014,кр/д (бв),450,21.12.2016,кр/д (бв),450,29.03.2019,кр/д (бв),450
4,13.08.2015,кр/д (бв),450,"21,06,2017",кр/д (бв),450,11.10.2022,кр/д (бв),450
5,30.10.2015,кр/д (бв),450,,,,,,


# pytesseract для текста + EasyOCR для недостающих

In [9]:
def img2table(img_path, output_path):
    # Считывание изображения
    img = Image.open(img_path)
    img = np.array(img)
    if img.ndim == 3 and img.shape[2] == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Присвоение изображению порогового значения в виде двоичного изображения
    img_bin = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 21, 10)

    # Ширина ядра как 100-я часть общей ширины
    kernel_len = np.array(img).shape[1] // 100

    # Определение вертикального и горизонтального ядер
    ver_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, kernel_len))
    hor_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_len, 1))

    # Ядро размером 2x2
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))

    # Обнаружение вертикальных и горизонтальных линий
    image_1 = cv2.erode(img_bin, ver_kernel, iterations=3)
    vertical_lines = cv2.dilate(image_1, ver_kernel, iterations=3)

    image_2 = cv2.erode(img_bin, hor_kernel, iterations=3)
    horizontal_lines = cv2.dilate(image_2, hor_kernel, iterations=3)

    # Объединение горизонтальных и вертикальных линий в новом изображении
    img_vh = cv2.addWeighted(vertical_lines, 0.5, horizontal_lines, 0.5, 0.0)

    # Размывание и установление порогового значения
    img_vh = cv2.erode(~img_vh, kernel, iterations=2)
    _, img_vh = cv2.threshold(img_vh, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

    # Обнаружение контуров
    contours, _ = cv2.findContours(img_vh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Преобразование tuple в list
    contours = list(contours)
    contours.reverse()

    # Подсчёт вертикальных линий (столбцов)
    num_vertical_lines = 1
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        
        if w > 20 and h > 20 and w < 500 and h < 500:
            num_vertical_lines = int(img.shape[1] / w)
            break
    
    texts = []
    # Итерация по каждому контуру (ячейке)
    for contour in contours:
        
        # Получение координат ограничивающего прямоугольника вокруг контура
        x, y, w, h = cv2.boundingRect(contour)
        if w > 20 and h > 20 and w < 500 and h < 500:
            cell_image = img[y:y + h, x:x + w]
        
            # Получение текста с ячейки
            text = pytesseract.image_to_string(cell_image, lang='rus')

            if text == '':
                results = reader.readtext(cell_image)
                if len(results) > 0:
                    text = results[0][1]
        
            # Добавление текста в список
            texts.append(text.replace('|', '').replace('_', ''))
    
    # Разделение списка по колучиству столбцов
    texts = [texts[i:i+num_vertical_lines] for i in range(0, len(texts), num_vertical_lines)]
    
    # Создание датафрейма
    try:
        df = pd.DataFrame(data=texts[1:], columns=texts[0])
    except:
        df = pd.DataFrame(columns=texts)
    df.dropna(inplace=True)
    df = df.applymap(lambda x: x.replace('\n', ''))
    df = df.rename(columns=lambda x: re.sub(r'\s+', ' ', x.replace('\n', ' ')))
    
    # Сохранение
    df.to_csv(output_path, index=False)

    return df

In [10]:
easy_empt_path = path + 'easy_empt.csv'

In [11]:
%%time
img2table(img_path, easy_empt_path)

CPU times: total: 7.48 s
Wall time: 5.34 s


,Дата,Вид дон-ва,Кол-во,Дата,Вид дон-ва,Кол-во,Дата,Вид дон-ва,Кол-во
0,,,,,,,,,
1,14.02.2006,кр/д (бв),420,15.07.2016,кр/д (бв),450,04.08.2018,кр/д (бв),450
2,11.06.2014,кр/д (бв),350,11.10.2016.,кр/д (бв),450,26.12.2018,кр/д (бв),450
3,30.10.2014,кр/д (бв),450,21.12.2016,кр/д (бв),450,29.03.2019,кр/д (бв),450
4,13.08.2015,кр/д (бв),450,21.06.2017,кр/д (бв),450,11.10.2022,кр/д (бв),450
5,30.10.2015,кр/д (бв),450,,,,,,


# easyocr

In [12]:
def img2table(img_path):
    # Считывание изображения
    img = Image.open(img_path)
    img = np.array(img)
    if img.ndim == 3 and img.shape[2] == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Присвоение изображению порогового значения в виде двоичного изображения
    img_bin = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 21, 10)

    # Ширина ядра как 100-я часть общей ширины
    kernel_len = np.array(img).shape[1] // 100

    # Определение вертикального и горизонтального ядер
    ver_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, kernel_len))
    hor_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_len, 1))

    # Ядро размером 2x2
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))

    # Обнаружение вертикальных и горизонтальных линий
    image_1 = cv2.erode(img_bin, ver_kernel, iterations=3)
    vertical_lines = cv2.dilate(image_1, ver_kernel, iterations=3)

    image_2 = cv2.erode(img_bin, hor_kernel, iterations=3)
    horizontal_lines = cv2.dilate(image_2, hor_kernel, iterations=3)

    # Объединение горизонтальных и вертикальных линий в новом изображении
    img_vh = cv2.addWeighted(vertical_lines, 0.5, horizontal_lines, 0.5, 0.0)

    # Размывание и установление порогового значения
    img_vh = cv2.erode(~img_vh, kernel, iterations=2)
    _, img_vh = cv2.threshold(img_vh, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

    # Обнаружение контуров
    contours, _ = cv2.findContours(img_vh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Преобразование tuple в list
    contours = list(contours)
    contours.reverse()

    # Подсчёт вертикальных линий (столбцов)
    num_vertical_lines = 1
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        
        if w > 20 and h > 20 and w < 500 and h < 500:
            num_vertical_lines = int(img.shape[1] / w)
            break
    
    texts = []
    # Итерация по каждому контуру (ячейке)
    for contour in contours:
        
        # Получение координат ограничивающего прямоугольника вокруг контура
        x, y, w, h = cv2.boundingRect(contour)
        if w > 20 and h > 20 and w < 500 and h < 500:
            cell_image = img[y:y + h, x:x + w]
        
            # Получение текста с ячейки
            text = reader.readtext(cell_image)
            if len(text) > 0:
                text = text[0][1]
            else:
                text = ''
        
            # Добавление текста в список
            texts.append(text.replace('|', '').replace('_', ''))
    
    # Разделение списка по колучиству столбцов
    texts = [texts[i:i+num_vertical_lines] for i in range(0, len(texts), num_vertical_lines)]
    
    # Создание датафрейма
    try:
        df = pd.DataFrame(data=texts[1:], columns=texts[0])
    except:
        df = pd.DataFrame(columns=texts)
    df.dropna(inplace=True)
    df = df.applymap(lambda x: x.replace('\n', ''))
    df = df.rename(columns=lambda x: re.sub(r'\s+', ' ', x.replace('\n', ' ')))

    return df

In [13]:
%%time
img2table(img_path)

CPU times: total: 16.8 s
Wall time: 4.75 s


,Дата,ДОН-ва,Кол-во,Дата,Вид,Кол-во,Дата,Вид,Кол-во
0,,,,,,,,,
1,14.02.2006,кр/д (бв),420,15.07.2016,крад (бв),450,04.08.2018,крад (бв),450
2,11.06.2014,крад (бв),350,11.10.2016,крад (бв),450,"26.12,2018",кр/д (бв),450
3,30.10.2014,кр/д (бв),450,21.12.2016,крад (бв),450,29.03.2019,кр/д (бв),450
4,13.08.2015,кр/д (бв),450,"21,06,2017",крад (бв),450,11.10.2022,кр/д (бв),450
5,30.10.2015,кр/д (бв),450,,,,,,


# pytesseract

In [14]:
def img2table(img_path, output_path):
    # Считывание изображения
    img = Image.open(img_path)
    img = np.array(img)
    if img.ndim == 3 and img.shape[2] == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Присвоение изображению порогового значения в виде двоичного изображения
    img_bin = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 21, 10)

    # Ширина ядра как 100-я часть общей ширины
    kernel_len = np.array(img).shape[1] // 100

    # Определение вертикального и горизонтального ядер
    ver_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, kernel_len))
    hor_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_len, 1))

    # Ядро размером 2x2
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))

    # Обнаружение вертикальных и горизонтальных линий
    image_1 = cv2.erode(img_bin, ver_kernel, iterations=3)
    vertical_lines = cv2.dilate(image_1, ver_kernel, iterations=3)

    image_2 = cv2.erode(img_bin, hor_kernel, iterations=3)
    horizontal_lines = cv2.dilate(image_2, hor_kernel, iterations=3)

    # Объединение горизонтальных и вертикальных линий в новом изображении
    img_vh = cv2.addWeighted(vertical_lines, 0.5, horizontal_lines, 0.5, 0.0)

    # Размывание и установление порогового значения
    img_vh = cv2.erode(~img_vh, kernel, iterations=2)
    _, img_vh = cv2.threshold(img_vh, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

    # Обнаружение контуров
    contours, _ = cv2.findContours(img_vh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Преобразование tuple в list
    contours = list(contours)
    contours.reverse()

    # Подсчёт вертикальных линий (столбцов)
    num_vertical_lines = 1
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        
        if w > 20 and h > 20 and w < 500 and h < 500:
            num_vertical_lines = int(img.shape[1] / w)
            break
    
    texts = []
    # Итерация по каждому контуру (ячейке)
    for contour in contours:
        
        # Получение координат ограничивающего прямоугольника вокруг контура
        x, y, w, h = cv2.boundingRect(contour)
        if w > 20 and h > 20 and w < 500 and h < 500:
            cell_image = img[y:y + h, x:x + w]
        
            # Получение текста с ячейки
            text = pytesseract.image_to_string(cell_image, lang='rus')
        
            # Добавление текста в список
            texts.append(text.replace('|', '').replace('_', ''))
    
    # Разделение списка по колучиству столбцов
    texts = [texts[i:i+num_vertical_lines] for i in range(0, len(texts), num_vertical_lines)]
    
    # Создание датафрейма
    try:
        df = pd.DataFrame(data=texts[1:], columns=texts[0])
    except:
        df = pd.DataFrame(columns=texts)
    df.dropna(inplace=True)
    df = df.applymap(lambda x: x.replace('\n', ''))
    df = df.rename(columns=lambda x: re.sub(r'\s+', ' ', x.replace('\n', ' ')))
    
    # Сохранение
    df.to_csv(output_path, index=False)

    return df

In [15]:
tes_path = path + 'tes.csv'

In [16]:
%%time
img2table(img_path, tes_path)

CPU times: total: 844 ms
Wall time: 8.83 s


,Дата,Вид дон-ва,Кол-во,Дата,Вид дон-ва,Кол-во,Дата,Вид дон-ва,Кол-во
0,,,,,,,,,
1,14.02.2006,кр/д (бв),420,15.07.2016,кр/д (бв),450,04.08.2018,кр/д (бв),450
2,,кр/д (бв),350,11.10.2016.,кр/д (бв),450,26.12.2018,кр/д (бв),450
3,30.10.2014,кр/д (бв),450,21.12.2016,кр/д (бв),,29.03.2019,кр/д (бв),450
4,13.08.2015,кр/д (бв),,21.06.2017,кр/д (бв),450,11.10.2022,кр/д (бв),450
5,30.10.2015,кр/д (бв),,,,,,,
